<a href="https://colab.research.google.com/github/chefinha-tech/acad_control/blob/main/verificador_certificado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ssl
import socket
import datetime
import os

# --- Configurações de Alerta ---
DIAS_PARA_EXPIRAR_ALERTA = 30 # Notificar se faltarem X dias ou menos para o certificado expirar
ALERTA_LOG_FILE = "alerta_certificados.log" # Nome do arquivo de log onde os alertas serão gravados

def registrar_alerta_em_log(message):
    """
    Registra uma mensagem de alerta em um arquivo de log.
    """
    timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    log_entry = f"[{timestamp}] {message}\n"
    try:
        with open(ALERTA_LOG_FILE, "a") as f: # O 'a' significa append (adicionar ao final)
            f.write(log_entry)
        print(f"Alerta registrado em '{ALERTA_LOG_FILE}'")
    except Exception as e:
        print(f"Erro ao registrar alerta em arquivo: {e}")

def verificar_certificado_site(hostname, port=443):
    """
    Verifica a validade do certificado SSL/TLS de um site.
    Args:
        hostname (str): O nome do host (domínio) do site.
        port (int): A porta (padrão 443 para HTTPS).
    Returns:
        dict: Um dicionário com o status do certificado e informações.
    """
    context = ssl.create_default_context()
    try:
        with socket.create_connection((hostname, port)) as sock:
            with context.wrap_socket(sock, server_hostname=hostname) as ssock:
                cert = ssock.getpeercert()

                not_before_str = cert['notBefore']
                not_after_str = cert['notAfter']

                try:
                    def parse_cert_date(date_str):
                        if date_str.endswith(' GMT'):
                            dt_naive = datetime.datetime.strptime(date_str[:-4], '%b %d %H:%M:%S %Y')
                            return dt_naive.replace(tzinfo=datetime.timezone.utc)
                        else:
                            dt = datetime.datetime.strptime(date_str, '%b %d %H:%M:%S %Y %Z')
                            if dt.tzinfo is None:
                                dt = dt.replace(tzinfo=datetime.timezone.utc)
                            return dt

                    not_before = parse_cert_date(not_before_str)
                    not_after = parse_cert_date(not_after_str)

                except ValueError:
                    return {
                        "status": "Erro na data",
                        "hostname": hostname,
                        "detalhes": "Não foi possível parsear as datas de validade do certificado. Verifique o formato."
                    }

                current_time = datetime.datetime.now(datetime.timezone.utc)

                status = "Válido"
                dias_restantes = (not_after - current_time).days if current_time < not_after else None

                if current_time < not_before:
                    status = "Ainda não válido (data futura)"
                elif current_time > not_after:
                    status = "Ainda não válido (data futura)" # Corrigido para ser mais claro
                elif dias_restantes is not None and dias_restantes <= DIAS_PARA_EXPIRAR_ALERTA:
                    status = f"Válido, mas expira em {dias_restantes} dias"
                elif current_time > not_after: # Certificado expirou
                     status = "Expirado"


                return {
                    "status": status,
                    "hostname": hostname,
                    "not_before": not_before.strftime('%Y-%m-%d %H:%M:%S UTC'),
                    "not_after": not_after.strftime('%Y-%m-%d %H:%M:%S UTC'),
                    "dias_restantes": dias_restantes,
                    "issuer": dict(x[0] for x in cert['issuer'])
                }

    except ssl.SSLCertVerificationError as e:
        return {
            "status": "Erro de verificação SSL",
            "hostname": hostname,
            "detalhes": str(e)
        }
    except socket.gaierror:
        return {
            "status": "Erro de DNS",
            "hostname": hostname,
            "detalhes": "Não foi possível resolver o nome do host."
        }
    except ConnectionRefusedError:
        return {
            "status": "Conexão recusada",
            "hostname": hostname,
            "detalhes": "A conexão foi recusada pelo servidor. Porta incorreta ou firewall?"
        }
    except Exception as e:
        return {
            "status": "Erro inesperado",
            "hostname": hostname,
            "detalhes": str(e)
        }

# --- Como usar a automação ---
if __name__ == "__main__":
    sites_file = "sites.txt"
    sites_para_verificar = []

    if os.path.exists(sites_file):
        with open(sites_file, "r") as f:
            for line in f:
                site = line.strip()
                if site:
                    sites_para_verificar.append(site)
    else:
        print(f"Arquivo '{sites_file}' não encontrado. Criando um exemplo com sites padrão.")
        with open(sites_file, "w") as f:
            f.write("google.com\n")
            f.write("certifica.com.br\n")
            f.write("expired.badssl.com\n") # Para testar certificado expirado
            f.write("digicert.com\n") # Exemplo de site válido
        sites_para_verificar = ["google.com", "certifica.com.br", "expired.badssl.com", "digicert.com"]

    if not sites_para_verificar:
        print("Nenhum site encontrado para verificar.")
    else:
        print("Iniciando verificação de certificados SSL/TLS...\n")

        # Limpa o arquivo de log a cada nova execução, ou use "a" para manter o histórico
        # with open(ALERTA_LOG_FILE, "w") as f: # Use "w" para sobrescrever, "a" para adicionar
        #    f.write(f"--- Início da Verificação: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')} ---\n")

        for site in sites_para_verificar:
            print(f"Verificando: {site}")
            resultado = verificar_certificado_site(site)
            for key, value in resultado.items():
                print(f"  {key}: {value}")
            print("-" * 30)

            # Lógica para registrar alertas no arquivo de log
            alerta_message = None
            if resultado["status"] == "Expirado":
                alerta_message = f"ALERTA CRÍTICO: O certificado para {resultado['hostname']} está EXPIRADO desde {resultado['not_after']}!"
            elif resultado["status"].startswith("Válido, mas expira"):
                alerta_message = f"AVISO: O certificado para {resultado['hostname']} expira em {resultado['dias_restantes']} dias!"
            elif resultado["status"].startswith("Erro"):
                alerta_message = f"ERRO na Verificação: Problema ao verificar o certificado de {resultado['hostname']}: {resultado['detalhes']}"

            if alerta_message:
                registrar_alerta_em_log(alerta_message)

        print(f"\nVerificação concluída. Verifique o arquivo '{ALERTA_LOG_FILE}' no painel de arquivos para ver os alertas.")

In [ ]:
!pip install PyPDF2


In [ ]:
import PyPDF2
import io
import os

# Certifique-se de que este nome de arquivo corresponda ao nome do PDF que você carregou no Colab
pdf_file_name = "certificado graduação.pdf"

# --- Código para processar o PDF ---
try:
    if not os.path.exists(pdf_file_name):
        print(f"Erro: O arquivo '{pdf_file_name}' não foi encontrado no seu ambiente Colab.")
        print("Por favor, certifique-se de ter feito o upload do arquivo para o painel 'Arquivos' do Colab.")
    else:
        with open(pdf_file_name, 'rb') as file: # 'rb' significa leitura em modo binário
            pdf_reader = PyPDF2.PdfReader(file)

            print(f"Total de páginas no PDF: {len(pdf_reader.pages)}")

            signature_fields_found = 0

            # Verifica o /AcroForm no trailer do PDF (estrutura principal)
            if "/AcroForm" in pdf_reader.trailer and pdf_reader.trailer["/AcroForm"] is not None:
                acro_form = pdf_reader.trailer["/AcroForm"].get_object()

                if "/SigFlags" in acro_form:
                    print(f"O PDF possui sinalizadores de assinatura (SigFlags: {acro_form['/SigFlags']}), indicando potencial para assinaturas digitais.")

                if "/Fields" in acro_form:
                    print(f"Verificando campos de formulário para assinaturas...")
                    for field in acro_form["/Fields"]:
                        field_obj = field.get_object()
                        if "/FT" in field_obj and field_obj["/FT"] == "/Sig":
                            signature_fields_found += 1
                            print(f"  - Campo de assinatura digital encontrado: {field_obj.get('/T', 'Nome não disponível')}")

            # Também pode haver anotações de assinatura nas páginas
            for i, page in enumerate(pdf_reader.pages):
                if "/Annots" in page:
                    for annot in page["/Annots"]:
                        annot_obj = annot.get_object()
                        if "/Subtype" in annot_obj and annot_obj["/Subtype"] == "/Widget":
                            if "/FT" in annot_obj and annot_obj["/FT"] == "/Sig":
                                signature_fields_found += 1
                                print(f"  - Campo de assinatura digital (anotação) encontrado na página {i+1}: {annot_obj.get('/T', 'Nome não disponível')}")

            if signature_fields_found > 0:
                print(f"\nResumo: Foram encontrados {signature_fields_found} campos que parecem ser de assinatura digital neste PDF.")
                print("Para *validar* as assinaturas e extrair os certificados, precisaremos de um passo mais complexo, possivelmente com uma biblioteca mais especializada como 'pyhanko', pois PyPDF2 tem suporte limitado para validação criptográfica direta.")
            else:
                print("\nNenhum campo que parece ser de assinatura digital foi detectado diretamente neste PDF usando PyPDF2.")
                print("Note: Alguns PDFs podem ter assinaturas que não são representadas como campos de formulário /Sig, ou o PyPDF2 pode não detectá-las em todos os casos.")

except PyPDF2.errors.PdfReadError as e:
    print(f"Erro ao ler o PDF. O arquivo pode estar corrompido ou não é um PDF válido: {e}")
except Exception as e:
    print(f"Ocorreu um erro inesperado ao processar o PDF: {e}")

In [49]:
!pip install pyhanko[full]

In [50]:
import os
import datetime
# Importações específicas do pyhanko
from pyhanko.pdf_utils.reader import PdfFileReader # A classe correta do pyhanko
from pyhanko.sign import validation
from cryptography.x509 import SubjectAlternativeName, ExtensionNotFound
from cryptography.x509.oid import ExtensionOID
from cryptography.x509 import DNSName

# --- Configurações de Alerta (Mantenha do seu código original) ---
DIAS_PARA_EXPIRAR_ALERTA = 30
ALERTA_LOG_FILE = "alerta_certificados.log"

def registrar_alerta_em_log(message):
    """
    Registra uma mensagem de alerta em um arquivo de log.
    """
    timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    log_entry = f"[{timestamp}] {message}\n"
    try:
        with open(ALERTA_LOG_FILE, "a") as f:
            f.write(log_entry)
        print(f"Alerta registrado em '{ALERTA_LOG_FILE}'")
    except Exception as e:
        print(f"Erro ao registrar alerta em arquivo: {e}")

# Nome do arquivo PDF que você carregou
pdf_file_name = "certificado graduação.pdf"

# --- Código para processar o PDF com pyhanko ---
try:
    if not os.path.exists(pdf_file_name):
        print(f"Erro: O arquivo '{pdf_file_name}' não foi encontrado no seu ambiente Colab.")
        print("Por favor, certifique-se de ter feito o upload do arquivo para o painel 'Arquivos' do Colab.")
    else:
        print(f"Iniciando a verificação de assinaturas no PDF: {pdf_file_name}")

        # Abre o PDF para leitura usando o PdfFileReader do pyhanko
        with open(pdf_file_name, "rb") as f:
            reader = PdfFileReader(f) # Aqui estamos garantindo que é o PdfFileReader do pyhanko

            signatures = reader.get_signatures() # Este método é do pyhanko.PdfFileReader

            if not signatures:
                print("\nEste PDF não parece conter assinaturas digitais detectáveis pelo pyhanko.")
                print("Nota: A ausência de assinaturas detectáveis pode significar que o PDF não foi assinado digitalmente, ou a assinatura está em um formato incomum/inválido.")
                registrar_alerta_em_log(f"AVISO: PDF '{pdf_file_name}' não contém assinaturas digitais detectáveis.")
            else:
                print(f"\nAssinaturas digitais encontradas: {len(signatures)}")

                for i, sig_meta in enumerate(signatures):
                    print(f"\n--- Assinatura {i+1} ---")
                    print(f"  Nome do campo de assinatura: {sig_meta.field_name}")
                    print(f"  Página onde a assinatura está localizada: {sig_meta.on_page + 1}")

                    try:
                        validation_info = validation.validate_pdf_signature(sig_meta)

                        print(f"  Status da Assinatura (Integridade): {validation_info.validation_result.summary_dict['int_integrity']}")
                        print(f"  Validade da Assinatura (Cadeia de Confiança): {validation_info.validation_result.summary_dict['validity']}")

                        signer_cert = validation_info.signer_certs[0]

                        print("\n  --- Detalhes do Certificado do Assinante ---")
                        print(f"  Assinado por: {signer_cert.subject.rfc4514_string()}")
                        print(f"  Emitido por: {signer_cert.issuer.rfc4514_string()}")
                        print(f"  Válido de: {signer_cert.not_valid_before_utc.strftime('%Y-%m-%d %H:%M:%S UTC')}")
                        print(f"  Válido até: {signer_cert.not_valid_after_utc.strftime('%Y-%m-%d %H:%M:%S UTC')}")

                        current_time = datetime.datetime.now(datetime.timezone.utc)
                        dias_restantes = (signer_cert.not_valid_after_utc - current_time).days if current_time < signer_cert.not_valid_after_utc else 0

                        alerta_status_data = ""
                        if dias_restantes < 0:
                            print("  Status de validade da data: EXPIRADO")
                            alerta_status_data = f"CERTIFICADO EXPIRADO no PDF '{pdf_file_name}' (Assinatura {i+1})."
                        elif dias_restantes <= DIAS_PARA_EXPIRAR_ALERTA:
                            print(f"  Status de validade da data: Válido, mas expira em {dias_restantes} dias")
                            alerta_status_data = f"CERTIFICADO EXPIRA EM BREVE no PDF '{pdf_file_name}' ({dias_restantes} dias - Assinatura {i+1})."
                        else:
                            print(f"  Status de validade da data: Válido ({dias_restantes} dias restantes)")

                        try:
                            san_extension = signer_cert.extensions.get_extension_for_oid(ExtensionOID.SUBJECT_ALTERNATIVE_NAME)
                            if san_extension and isinstance(san_extension.value, SubjectAlternativeName):
                                dns_names = san_extension.value.get_values_for_type(DNSName)
                                if dns_names:
                                    print(f"  Nomes Alternativos (DNS SAN): {', '.join(dns_names)}")
                        except ExtensionNotFound:
                            pass

                        revocation_status = validation_info.validation_result.summary_dict['revocation_status']
                        print(f"  Status de Revogação (pyhanko): {revocation_status}")

                        alerta_revogacao = ""
                        if revocation_status != validation.RevocationInfoValidationResult.UNDEFINED and \
                           revocation_status != validation.RevocationInfoValidationResult.OCSP_RESPONSE_GOOD and \
                           revocation_status != validation.RevocationInfoValidationResult.CRL_GOOD:
                            alerta_revogacao = f"ALERTA DE REVOGAÇÃO: Certificado revogado ou status desconhecido no PDF '{pdf_file_name}' (Assinatura {i+1}): {revocation_status}"

                        if alerta_status_data:
                            registrar_alerta_em_log(alerta_status_data)
                        if alerta_revogacao:
                            registrar_alerta_em_log(alerta_revogacao)
                        if validation_info.validation_result.summary_dict['int_integrity'] != 'int_ok':
                             registrar_alerta_em_log(f"ALERTA: Integridade da assinatura comprometida no PDF '{pdf_file_name}' (Assinatura {i+1}): {validation_info.validation_result.summary_dict['int_integrity']}")
                        if validation_info.validation_result.summary_dict['validity'] != validation.V_RESID_SIG_VALID:
                             registrar_alerta_em_log(f"ALERTA: Validade da cadeia de confiança da assinatura inválida no PDF '{pdf_file_name}' (Assinatura {i+1}): {validation_info.validation_result.summary_dict['validity']}")

                    except Exception as val_e:
                        print(f"  Erro ao validar ou extrair informações da assinatura: {val_e}")
                        print("  Pode ser que a assinatura esteja corrompida, não seja válida, ou haja problemas com a cadeia de confiança.")
                        registrar_alerta_em_log(f"ERRO: Problema ao processar assinatura {i+1} no PDF '{pdf_file_name}': {val_e}")

except FileNotFoundError:
    print(f"Erro: O arquivo '{pdf_file_name}' não foi encontrado. Certifique-se de que ele foi carregado no ambiente do Colab.")
    registrar_alerta_em_log(f"ERRO: PDF '{pdf_file_name}' não encontrado.")
except Exception as e:
    print(f"Ocorreu um erro inesperado ao processar o PDF: {e}")
    registrar_alerta_em_log(f"ERRO GERAL ao processar PDF '{pdf_file_name}': {e}")

Iniciando a verificação de assinaturas no PDF: certificado graduação.pdf
Ocorreu um erro inesperado ao processar o PDF: 'PdfFileReader' object has no attribute 'get_signatures'
Alerta registrado em 'alerta_certificados.log'


In [ ]:
!pip install cryptography


In [ ]:
%%writefile sites.txt
google.com
certifica.com.br
globalsign.com
digicert.com
expired.badssl.com
sitequenaoexiste12345.com
mec.gov.br
emec.mec.gov.br